In [ ]:
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils, TopicAndPartition

配置`spark.streaming.stopGracefullyOnShutdown`为`true`

In [ ]:
spark = SparkSession.builder \
    .appName('test') \
    .config('spark.streaming.stopGracefullyOnShutdown', 'true') \
    .getOrCreate()
sc = spark.sparkContext
sql_ctx = SQLContext(sc)
ssc = StreamingContext(sc, 10)

实时处理函数

In [ ]:
def process_func(ts, rdd):
    # do something
    # offset commit
    return rdd

连接kafka 开启ssc

In [ ]:
kafka_params = {
    'bootstrap.servers': KAFKA_CONFIG['servers'],
    'group.id': KAFKA_CONFIG['group_id'],
    'auto.offset.reset': 'largest',
    'auto.commit.enable': 'false', # 测试了一下这个参数 是无效的 不会commit
}
dks = KafkaUtils.createDirectStream(ssc, [KAFKA_CONFIG['topics']], kafka_params) 

dks = dks.transform(process_func)
dks.pprint()

ssc.start()
ssc.awaitTermination()

使用`kill pid`的方式终止spark进程

据观察，两个批次后spark会关闭（参考链接说是跟spark.yarn.maxAppAttempts有关，但是我尝试了一下，并没有关系）

In [ ]:
!ps aux | grep spark | grep SparkSubmit | awk '{print $2}' | xargs kill -SIGTERM

注意：不可以使用`yarn application -kill app_id`的方式终止

可以参考：https://www.linkedin.com/pulse/how-shutdown-spark-streaming-job-gracefully-lan-jiang